In [2]:
conda install -c conda-forge pandas


Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
import numpy as np

# read csv document
df = pd.read_csv('London.csv')
df


,id,date,confirmed,deaths,recovered,tests,vaccines,people_vaccinated,people_fully_vaccinated,hosp,...,iso_alpha_3,iso_alpha_2,iso_numeric,iso_currency,key_local,key_google_mobility,key_apple_mobility,key_jhu_csse,key_nuts,key_gadm
0,4f87db06,2020-01-03,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,...,GBR,GB,826,GBP,E09000012,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,"London, England",NaN,UKI31,GBR.1.36.23_1
1,4f87db06,2020-01-04,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,...,GBR,GB,826,GBP,E09000012,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,"London, England",NaN,UKI31,GBR.1.36.23_1
2,4f87db06,2020-01-05,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,...,GBR,GB,826,GBP,E09000012,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,"London, England",NaN,UKI31,GBR.1.36.23_1
3,4f87db06,2020-01-06,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,...,GBR,GB,826,GBP,E09000012,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,"London, England",NaN,UKI31,GBR.1.36.23_1
4,4f87db06,2020-01-07,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,...,GBR,GB,826,GBP,E09000012,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,"London, England",NaN,UKI31,GBR.1.36.23_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310,4f87db06,2023-08-05,97830.0,NaN,NaN,2555175,NaN,192731.0,179814.0,NaN,...,GBR,GB,826,GBP,E09000012,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,"London, England",NaN,UKI31,GBR.1.36.23_1
1311,4f87db06,2023-08-06,97831.0,NaN,NaN,2555181,499307.0,192731.0,179814.0,NaN,...,GBR,GB,826,GBP,E09000012,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,"London, England",NaN,UKI31,GBR.1.36.23_1
1312,4f87db06,2023-08-07,97833.0,NaN,NaN,2555194,499299.0,192731.0,179814.0,NaN,...,GBR,GB,826,GBP,E09000012,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,"London, England",NaN,UKI31,GBR.1.36.23_1
1313,4f87db06,2023-08-08,97835.0,NaN,NaN,2555205,499331.0,192731.0,179814.0,NaN,...,GBR,GB,826,GBP,E09000012,ChIJ4Y3fTlUDdkgR0Gbsoi2uDgQ,"London, England",NaN,UKI31,GBR.1.36.23_1


In [20]:
# get non-empty death data
death_non_empty = df['deaths'].dropna()
daily_deaths = death_non_empty.diff().dropna()

In [21]:
n = len(daily_deaths)
Y = np.zeros(n)

# Simulate the process
for t in range(n):
    Y[t] = np.random.poisson(daily_deaths.iloc[t])

In [22]:
# Simulated expectation and variance
mu_lambda_t_simulated = np.mean(daily_deaths)
var_lambda_t_simulated = np.var(daily_deaths)
mu_Y_t_simulated = np.mean(Y)
var_Y_t_simulated = np.var(Y)

# Simulated covariance
cov_matrix = np.cov(Y[:-1], Y[1:])
cov_Y_t_simulated = cov_matrix[0, 1]

In [24]:
from scipy.optimize import fsolve

# define function
def equations(vars):
    x, y, z = vars
    eq1 = z/(1-x-y) - mu_Y_t_simulated
    eq2 = z/(1-x-y) * (1 + y**2/(1-(x+y)**2)) - var_Y_t_simulated
    eq3 = x * z/(1-x-y) * (y**2/(1-(x+y)**2)) + y * z/(1-x-y) * (1 + y**2/(1-(x+y)**2)) - cov_Y_t_simulated
    return [eq1, eq2, eq3]

# initail guess
initial_guess = [0.2, 0.63, 0.05]

# use fsolve to solve
solution = fsolve(equations, initial_guess)

print("solution: a = {}, b = {}, d = {}".format(solution[0], solution[1], solution[2]))


solution: a = 0.8850418453877095, b = 0.11161673669370197, d = 0.0023924823669165744


C:\anaconda\lib\site-packages\scipy\optimize\_minpack_py.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


In [17]:
x,y,z = solution
eq1 = z/(1-x-y) - mu_Y_t_simulated
eq2 = z/(1-x-y) * (1 + y**2/(1-(x+y)**2)) - var_Y_t_simulated
eq3 = x * z/(1-x-y) * (y**2/(1-(x+y)**2)) + y * z/(1-x-y) * (1 + y**2/(1-(x+y)**2)) - cov_Y_t_simulated
print(eq1, eq2, eq3)

1.0391687510491465e-13 2.9531932455029164e-13 2.915445662665661e-13
